In [1]:
import numpy as np
import pandas as pd

In [2]:
from pandas.tseries.offsets import Day, Week

In [3]:
stores = pd.read_parquet('/main/data/MVideo_2/stores.parquet')

In [4]:
nostores = pd.read_parquet('/main/data/MVideo_2/nostores.parquet')

In [5]:
st = np.setdiff1d(stores.Store_ID.values, nostores.Store_ID.values)

In [6]:
products = pd.read_parquet('/main/data/MVideo_2/skus.parquet')

In [7]:
noproducts = pd.read_parquet('/main/data/MVideo_2/noskus.parquet')

In [8]:
pr = np.setdiff1d(products.Product_ID.values, noproducts.Product_ID.values)

In [9]:
trans = pd.read_parquet('/main/data/MVideo_2/dailysales.parquet')

In [10]:
trans1 = trans[
    (trans.STORE_ID.isin(st)) &
    (trans.PRODUCT_ID.isin(pr))
].copy()

In [11]:
trans1.drop(columns = ['PROMOTION_ID', 'UNITS_SOLD', 'REVENUE_INCL_TAX'], inplace = True)

In [12]:
#trans1 = trans1[(trans1.DATE > '2016-07-03')]

In [13]:
#trans1 = trans1[(trans1.DATE <= '2018-07-08')]

In [14]:
len(set(list(zip(trans1.STORE_ID, trans1.PRODUCT_ID))))

115269

In [15]:
week = trans1[['DATE', 'STORE_ID', 'PRODUCT_ID']]

In [16]:
week1 = week.copy()

In [17]:
week1.DATE = week1.DATE.where(week1.DATE == ((week1.DATE + Week(weekday=6)) - Week()), week1.DATE + Week(weekday=6))

In [18]:
week1.DATE.max()

Timestamp('2018-07-15 00:00:00')

In [19]:
week1.drop_duplicates(inplace = True)

In [20]:
week1.shape

(801526, 3)

In [21]:
week1min = week1.groupby(['STORE_ID', 'PRODUCT_ID'])[['DATE']].min().reset_index()

In [22]:
week1max = week1.groupby(['STORE_ID', 'PRODUCT_ID'])[['DATE']].max().reset_index()

In [23]:
weekrange = pd.merge(week1min, week1max, on = ['STORE_ID', 'PRODUCT_ID'])

In [24]:
weekrange.rename(columns = {'DATE_x': 'DATE_MIN', 'DATE_y': 'DATE_MAX'}, inplace = True)

Заполняем хвосты вправо в тесте

In [25]:
weekrange.DATE_MAX = weekrange.DATE_MAX.where(weekrange.DATE_MAX < '2018-05-20', pd.Timestamp('2018-07-08'))

In [26]:
weekrange.DATE_MIN += Week(12)

In [27]:
week2 = pd.merge(weekrange[['STORE_ID', 'PRODUCT_ID']].assign(key = 0),
                 pd.DataFrame({'DATE': pd.date_range(start = weekrange.DATE_MIN.min(),
                                                     end = weekrange.DATE_MAX.max(),
                                                     freq = '7D')}).assign(key = 0),
                 on = 'key').drop('key', axis = 1)

In [28]:
week2.shape

(10835286, 3)

In [29]:
week3 = week2.merge(weekrange, how = 'left', on = ['STORE_ID', 'PRODUCT_ID'])

In [30]:
week4 = week3[
    (week3.DATE >= week3.DATE_MIN) &
    (week3.DATE <= week3.DATE_MAX)
].copy()

In [31]:
week4.drop(columns = ['DATE_MIN', 'DATE_MAX'], inplace = True)

In [32]:
week4.shape

(1971462, 3)

In [33]:
week4.DATE -= Day(7)

In [34]:
week4['ID'] = week4.STORE_ID.astype('str').values + ' ' + week4.PRODUCT_ID.astype('str').values

In [35]:
week4[week4.DATE < '2018-05-13'].shape

(1806839, 4)

In [36]:
week4[week4.DATE >= '2018-05-13'].shape

(164623, 4)

In [37]:
week41 = week4[week4.DATE >= '2018-05-13'].copy()

In [38]:
id1 = week41.ID.drop_duplicates().values

In [39]:
len(id1)

22086

In [40]:
week42 = pd.read_csv('/main/data/MVideo_2/id.csv')

In [41]:
id2 = week42.drop_duplicates()['0'].values

In [42]:
len(id2)

21885

In [43]:
week43 = week41[week41.ID.isin(set(id2))]

In [44]:
week43.shape

(163122, 4)

In [45]:
week44 = week43.drop(columns = ['ID']).copy()

In [46]:
week44.STORE_ID = week44.STORE_ID.astype('str').str[1:].astype('int')

In [47]:
week44[['STORE_ID', 'PRODUCT_ID']] = week44[['STORE_ID', 'PRODUCT_ID']].astype('int32')

In [48]:
week44[['STORE_ID', 'PRODUCT_ID']] = week44[['STORE_ID', 'PRODUCT_ID']].astype('category')

In [49]:
week44.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 163122 entries, 3000 to 10833687
Data columns (total 3 columns):
STORE_ID      163122 non-null category
PRODUCT_ID    163122 non-null category
DATE          163122 non-null datetime64[ns]
dtypes: category(2), datetime64[ns](1)
memory usage: 3.2 MB


In [50]:
week44.to_parquet('/main/data/MVideo_2/crosstest.parquet')

In [51]:
week45 = week4[(week4.DATE > '2016-09-25')].copy()

In [52]:
week45.shape

(1964993, 4)

In [53]:
week46 = week45[week45.ID.isin(set(id1))].copy()

In [54]:
week46.drop(columns = ['ID'], inplace = True)

In [55]:
week46.STORE_ID = week46.STORE_ID.astype('str').str[1:].astype('int')

In [56]:
week46[['STORE_ID', 'PRODUCT_ID']] = week46[['STORE_ID', 'PRODUCT_ID']].astype('int32')

In [57]:
week46[['STORE_ID', 'PRODUCT_ID']] = week46[['STORE_ID', 'PRODUCT_ID']].astype('category')

In [58]:
week46.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 752281 entries, 2258 to 10833687
Data columns (total 3 columns):
STORE_ID      752281 non-null category
PRODUCT_ID    752281 non-null category
DATE          752281 non-null datetime64[ns]
dtypes: category(2), datetime64[ns](1)
memory usage: 14.4 MB


In [59]:
week46.to_parquet('/main/data/MVideo_2/crosstrain.parquet')